In [257]:
import pandas as pd
import numpy as np
import json
import re
from collections import defaultdict
from scipy import sparse
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
eda = pd.read_csv('../WebScraping/edaru.csv')

In [21]:
ingredients = {}
for ings in eda.loc[:, 'ingredients'].values:
    for i in eval(ings):
        ingredients[eval(i)['id']] = eval(i)['name']

In [22]:
len(ingredients)

775

In [25]:
with open('ingredients.json', 'w') as f:
    json.dump(ingredients, f)

In [110]:
eda.head(4)

,img,title,link,stepbystep,cuisine,portions,time,ingredients,favourite,likes,dislikes,us,eu,ru,minutes
0,https://img05.rl0.ru/eda/c285x285i/s1.eda.ru/P...,Брауни (brownie),https://eda.ru/recepty/vypechka-deserty/brauni...,1,Американская кухня,6,40 минут,"['{""id"": 14797, ""name"": ""Темный шоколад"", ""amo...",25312,2996,247,1,0,0,40
1,https://img01.rl0.ru/eda/c285x285i/s2.eda.ru/P...,Цветаевский яблочный пирог,https://eda.ru/recepty/vypechka-deserty/cvetae...,1,Европейская кухня,4,0,"['{""id"": 13447, ""name"": ""Яблоко"", ""amount"": ""1...",15207,1960,190,0,1,0,0
2,https://img05.rl0.ru/eda/c285x285i/s1.eda.ru/P...,Классическая шарлотка,https://eda.ru/recepty/vypechka-deserty/klassi...,1,Русская кухня,12,35 минут,"['{""id"": 13410, ""name"": ""Сахар"", ""amount"": ""1 ...",18829,1948,223,0,0,1,35
3,https://img09.rl0.ru/eda/c285x285i/s2.eda.ru/P...,Тонкие блины на молоке,https://eda.ru/recepty/vypechka-deserty/tonkie...,1,NaN,4,40 минут,"['{""id"": 13458, ""name"": ""Пшеничная мука"", ""amo...",14250,1707,288,0,0,0,40


In [34]:
eda['us'] = np.where(eda.cuisine.str.contains('Американская', na=False), 1, 0) # na=False returns 0 when nan
eda.us = eda.us.astype('category')

In [36]:
eda['eu'] = np.where(eda.cuisine.str.contains('Европейская', na=False), 1, 0) # na=False returns 0 when nan
eda.us = eda.us.astype('category')
eda['ru'] = np.where(eda.cuisine.str.contains('Русская', na=False), 1, 0) # na=False returns 0 when nan
eda.us = eda.us.astype('category')

In [42]:
eda.stepbystep = eda.stepbystep.astype('category')

In [63]:
print(f'out of {eda.shape[0]} recipes, {eda.cuisine.isna().sum()} \
({round(100*eda.cuisine.isna().sum()/eda.shape[0], 1)}%) are missing cuisine')
print(f'american: {eda[eda.us == 1].shape[0]}\n\
european: {eda[eda.eu == 1].shape[0]}\n\
russian: {eda[eda.ru == 1].shape[0]}')
print(f'step by step: {eda[eda.stepbystep == 1].shape[0]}')
print(f'{eda.time.isna().sum()} missing time') # use average? median? 

out of 1769 recipes, 446 (25.2%) are missing cuisine
american: 141
european: 375
russian: 286
step by step: 85
164 missing time


In [70]:
eda.time = eda.time.fillna(0)

In [105]:
minutes = []
hour = re.compile('час')
mins = re.compile('минут')
for time in eda.loc[:, 'time'].values:
    if time == 0:
        minutes.append(0)
        
    else:
        
        # if end with hours
        if hour.search(time.split(' ')[-1]):
            t = int(time.split(' ')[-2])*60
            minutes.append(t)
            
        
        # if ends with minutes
        elif mins.search(time.split(' ')[-1]):
            t = int(time.split(' ')[-2])
        
            # check if contains hours
            try:
                t += int(time.split(' ')[-4])*60
                minutes.append(t)
            except IndexError:
                minutes.append(t)
                
eda['minutes'] = minutes

In [111]:
edacopy = eda.loc[:, ['stepbystep', 'cuisine', 'portions', 'ingredients', 'favourite', 'likes', 'dislikes', 'minutes']]

In [112]:
edacopy.head()

,stepbystep,cuisine,portions,ingredients,favourite,likes,dislikes,minutes
0,1,Американская кухня,6,"['{""id"": 14797, ""name"": ""Темный шоколад"", ""amo...",25312,2996,247,40
1,1,Европейская кухня,4,"['{""id"": 13447, ""name"": ""Яблоко"", ""amount"": ""1...",15207,1960,190,0
2,1,Русская кухня,12,"['{""id"": 13410, ""name"": ""Сахар"", ""amount"": ""1 ...",18829,1948,223,35
3,1,NaN,4,"['{""id"": 13458, ""name"": ""Пшеничная мука"", ""amo...",14250,1707,288,40
4,0,NaN,4,"['{""id"": 13418, ""name"": ""Яйцо куриное"", ""amoun...",12575,1489,253,60


In [132]:
s = set()
for ings in edacopy.loc[:, 'ingredients'].values:
    for i in eval(ings):
        s.add(' '.join(eval(i)['amount'].split(' ')[1:]))
s

{'',
 'банка',
 'банки',
 'веточки',
 'вкусу',
 'г',
 'головка',
 'головки',
 'головок',
 'зубчик',
 'зубчика',
 'зубчиков',
 'кг',
 'кончике ножа',
 'куска',
 'кусков',
 'кусок',
 'л',
 'мл',
 'пучка',
 'пучок',
 'стакан',
 'стакана',
 'стаканов',
 'стебель',
 'стеблей',
 'стебля',
 'столовая ложка',
 'столовой ложки',
 'столовые ложки',
 'столовых ложек',
 'чайная ложка',
 'чайной ложки',
 'чайные ложки',
 'чайных ложек',
 'штук',
 'штука',
 'штуки'}

In [122]:
for i in ingredients:
    edacopy[str(i)] = 0

In [194]:
edacopy.shape

(1769, 783)

## Веса продуктов

яйцо = 50г

яблоко = 175г

банан = 200г

стакан сахара/муки = 160г

столовая ложка = 20г

чайная ложка = 5г

банка сметаны = 200г

кусок = 200г

кусок шоколада = 50г

головка лука = 85г

головка чеснока = 45г

зубчик = 10г

стебель = 60г

по вкусу = 0г

на кончике ножа = 0г

щепотка = 0г

мл = г

л = 1000г

In [217]:
# for each row extract ingredients
for n in range(edacopy.shape[0]):
    for i in eval(edacopy.loc[n, 'ingredients']):

        # if grams or ml
        if eval(i)['amount'].split(' ')[-1] == 'г' or eval(i)['amount'].split(' ')[-1] == 'мл':
            try:
                value = int(eval(i)['amount'].split(' ')[-2])
            # has a comma
            except ValueError:
                try:
                    value = int(''.join(eval(i)['amount'].split(' ')[-2].split(',')))
                # a stupid fraction of a gram
                except ValueError:
                    value = 0

        # if kg or l
        elif eval(i)['amount'].split(' ')[-1] == 'кг' or eval(i)['amount'].split(' ')[-1] == 'л':
            try:
                value = int(eval(i)['amount'].split(' ')[-2])*1000
            # has a comma
            except ValueError:
                try:
                    value = int(''.join(eval(i)['amount'].split(' ')[-2].split(',')))*1000
                # a stupid fraction of a gram
                except ValueError:
                    value = 500

        # if an egg
        elif re.compile('йцо').search(eval(i)['name']) or re.compile('желт').search(eval(i)['name'])\
                or re.compile('белок').search(eval(i)['name']):
            value =  int(eval(i)['amount'][0])*50

        # if a banana
        elif re.compile('анан').search(eval(i)['name']):
            try:
                value = int(eval(i)['amount'][0])*200
            # a stupid fraction 
            except ValueError:
                value = 100

        # if an apple:
        elif re.compile('блоко').search(eval(i)['name']):
            try:
                value = int(eval(i)['amount'][0])*175
            # a stupid fraction
            except ValueError:
                value =  100

        # if a glass
        elif re.compile('стакан').search(eval(i)['amount']):
            try:
                value = int(eval(i)['amount'][0])*160
            # a stupid fraction
            except ValueError:
                value = 100

        # table spoon
        elif re.compile('стол').search(eval(i)['amount']):
            try:
                value = int(eval(i)['amount'][0])*20
            # a stupid fraction
            except ValueError:
                value = 10

        # tea spoon
        elif re.compile('чайн').search(eval(i)['amount']):
            try:
                 value = int(eval(i)['amount'][0])*5
            # a stupid fraction
            except ValueError:
                value = 3

        # can
        elif re.compile('банк').search(eval(i)['amount']):
            try:
                value = int(eval(i)['amount'][0])*200
            # has a comma
            except ValueError:
                try:
                    value = float('.'.join(eval(i)['amount'].split(' ')[0].split(',')))*200
                # a stupid fraction
                except ValueError:
                     value = 100

        # по вкусу
        elif re.compile('вкус').search(eval(i)['amount']) or re.compile('нож').search(eval(i)['amount'])\
                or re.compile('щепотка').search(eval(i)['amount']):
            value = 0

        # chocolate
        elif re.compile('околад').search(eval(i)['name']):
            value = int(eval(i)['amount'][0])*20

        # остальные куски
        elif re.compile('кус').search(eval(i)['amount']):
            value = int(eval(i)['amount'][0])*200 

        # зубчики
        elif re.compile('зубч').search(eval(i)['amount']):
            value = int(eval(i)['amount'][0])*10

        # головка лука
        elif re.compile('головк').search(eval(i)['amount']) and re.compile('ук').search(eval(i)['name']):
            try:
                value = int(eval(i)['amount'][0])*85 
            except ValueError:
                value = 40

        # головка чеснока
        elif re.compile('головк').search(eval(i)['amount']) and re.compile('еснок').search(eval(i)['name']):
            try:
                value = int(eval(i)['amount'][0])*45
            except ValueError:
                value = 20

         # стебли веточки
        elif re.compile('стеб').search(eval(i)['amount']) or re.compile('веточ').search(eval(i)['amount']):
            try:
                value = int(eval(i)['amount'][0])*20
            except ValueError:
                value = 10

        # ваниль корица печенье фундук пучки головки
        elif re.compile('аниль').search(eval(i)['name']) or re.compile('ориц').search(eval(i)['name']) or\
            re.compile('ечень').search(eval(i)['name']) or re.compile('ундук').search(eval(i)['name']) or\
            re.compile('пуч').search(eval(i)['amount']) or re.compile('голов').search(eval(i)['amount']):
            try:
                value = int(eval(i)['amount'][0])*20
            except ValueError:
                value = 10

        # remainig items
        elif re.compile('штук').search(eval(i)['amount']):
            try:
                value = int(eval(i)['amount'].split(' ')[0])*150
            except ValueError:
                try:
                    value = int(''.join(eval(i)['amount'].split(' ')[0].split(',')))*150
                # a stupid fraction of a gram
                except ValueError:
                    value = 100

        else:
            print(i)

        edacopy.loc[n, str(eval(i)['id'])] = value

In [222]:
ings_index = {}
num = 0
for i in list(edacopy)[8:]:
    ings_index[i] = num
    num += 1

In [225]:
with open('ings_index.json', 'w') as f:
    json.dump(ings_index, f)

In [232]:
edacopy = edacopy.drop(columns=['stepbystep', 'cuisine', 'portions', 'ingredients', 'favourite', 'likes', 'dislikes'])

In [234]:
# convert to condenced sparse row
edamtx = sparse.csr_matrix(edacopy)

In [235]:
scores = []
for n_clusters in range(2, 10):
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(edamtx)
    centers = clusterer.cluster_centers_

    score = silhouette_score(edamtx, preds, metric='euclidean')
    scores.append(score)
    print (f"For n_clusters = {n_clusters}, silhouette score is {score}")

For n_clusters = 2, silhouette score is 0.9986780356039926
For n_clusters = 3, silhouette score is 0.9900576322013934
For n_clusters = 4, silhouette score is 0.9897638337395733
For n_clusters = 5, silhouette score is 0.9311105511173017
For n_clusters = 6, silhouette score is 0.9346641086563133
For n_clusters = 7, silhouette score is 0.9275687956283898
For n_clusters = 8, silhouette score is 0.9358298233628269
For n_clusters = 9, silhouette score is 0.9359834698585056


In [236]:
for n_clusters in range(11, 20):
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(edamtx)
    centers = clusterer.cluster_centers_

    score = silhouette_score(edamtx, preds, metric='euclidean')
    scores.append(score)
    print (f"For n_clusters = {n_clusters}, silhouette score is {score}")

For n_clusters = 11, silhouette score is 0.9075280580115669
For n_clusters = 12, silhouette score is 0.919148868602606
For n_clusters = 13, silhouette score is 0.10792053791584033
For n_clusters = 14, silhouette score is 0.1105203328462498
For n_clusters = 15, silhouette score is 0.1076102057264956
For n_clusters = 16, silhouette score is 0.08690741302674225
For n_clusters = 17, silhouette score is 0.10632028302048385
For n_clusters = 18, silhouette score is 0.10249095159997604
For n_clusters = 19, silhouette score is 0.079687820500544


In [277]:
fig = plt.figure(figsize=(100, 75))
linkage = hierarchy.linkage(edacopy)
cutree = hierarchy.cut_tree(linkage, height=10000)

<Figure size 7200x5400 with 0 Axes>

In [278]:
np.unique(cutree)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])